Importando bibliotecas necessárias 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, lower
from pyspark.sql import functions as fn
import requests
import zipfile
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


Iniciando e configurando a sparkSession 

In [2]:

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.execution.arrow.enabled", True)

Fazendo o request da base de dados publica e descompactando os arquivos

In [3]:
request = requests.get('https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip', verify = False)
with open('../data/microdados.zip', 'wb') as f:
    f.write(request.content)
with zipfile.ZipFile('../data/microdados.zip', 'r') as zip_ref:
    zip_ref.extractall('../data')

Lendo os arquivo em csv

In [4]:


df = spark.read.csv("../data/Microdados do Censo da Educaç╞o Superior 2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV", header=True, sep=';', inferSchema = True)

df.printSchema()

root
 |-- NU_ANO_CENSO: integer (nullable = true)
 |-- NO_REGIAO_IES: string (nullable = true)
 |-- CO_REGIAO_IES: integer (nullable = true)
 |-- NO_UF_IES: string (nullable = true)
 |-- SG_UF_IES: string (nullable = true)
 |-- CO_UF_IES: integer (nullable = true)
 |-- NO_MUNICIPIO_IES: string (nullable = true)
 |-- CO_MUNICIPIO_IES: integer (nullable = true)
 |-- IN_CAPITAL_IES: integer (nullable = true)
 |-- NO_MESORREGIAO_IES: string (nullable = true)
 |-- CO_MESORREGIAO_IES: integer (nullable = true)
 |-- NO_MICRORREGIAO_IES: string (nullable = true)
 |-- CO_MICRORREGIAO_IES: integer (nullable = true)
 |-- TP_ORGANIZACAO_ACADEMICA: integer (nullable = true)
 |-- TP_CATEGORIA_ADMINISTRATIVA: integer (nullable = true)
 |-- NO_MANTENEDORA: string (nullable = true)
 |-- CO_MANTENEDORA: integer (nullable = true)
 |-- CO_IES: integer (nullable = true)
 |-- NO_IES: string (nullable = true)
 |-- SG_IES: string (nullable = true)
 |-- DS_ENDERECO_IES: string (nullable = true)
 |-- DS_NUMERO_

Deixando os nomes colunas com letras minúsculas

In [5]:
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

Salvado os arquivos em parquet (Não consegui salvar diretamente na pasta data, então salvei fora depois movi os arquivos manualmente)

In [6]:
df.write.format('parquet').mode('overwrite').save('../microdados_parquet')

Lendo os arquivos em parquet

In [7]:
df = spark.read.parquet("../microdados_parquet", header=True, sep=';', inferSchema = True)
df.printSchema()

root
 |-- nu_ano_censo: integer (nullable = true)
 |-- no_regiao_ies: string (nullable = true)
 |-- co_regiao_ies: integer (nullable = true)
 |-- no_uf_ies: string (nullable = true)
 |-- sg_uf_ies: string (nullable = true)
 |-- co_uf_ies: integer (nullable = true)
 |-- no_municipio_ies: string (nullable = true)
 |-- co_municipio_ies: integer (nullable = true)
 |-- in_capital_ies: integer (nullable = true)
 |-- no_mesorregiao_ies: string (nullable = true)
 |-- co_mesorregiao_ies: integer (nullable = true)
 |-- no_microrregiao_ies: string (nullable = true)
 |-- co_microrregiao_ies: integer (nullable = true)
 |-- tp_organizacao_academica: integer (nullable = true)
 |-- tp_categoria_administrativa: integer (nullable = true)
 |-- no_mantenedora: string (nullable = true)
 |-- co_mantenedora: integer (nullable = true)
 |-- co_ies: integer (nullable = true)
 |-- no_ies: string (nullable = true)
 |-- sg_ies: string (nullable = true)
 |-- ds_endereco_ies: string (nullable = true)
 |-- ds_numero_

Criando uma temp view para fazer consultas em SQL

In [8]:
df.createOrReplaceTempView("microdados_ies")

In [9]:
df

nu_ano_censo,no_regiao_ies,co_regiao_ies,no_uf_ies,sg_uf_ies,co_uf_ies,no_municipio_ies,co_municipio_ies,in_capital_ies,no_mesorregiao_ies,co_mesorregiao_ies,no_microrregiao_ies,co_microrregiao_ies,tp_organizacao_academica,tp_categoria_administrativa,no_mantenedora,co_mantenedora,co_ies,no_ies,sg_ies,ds_endereco_ies,ds_numero_endereco_ies,ds_complemento_endereco_ies,no_bairro_ies,nu_cep_ies,qt_tec_total,qt_tec_fundamental_incomp_fem,qt_tec_fundamental_incomp_masc,qt_tec_fundamental_comp_fem,qt_tec_fundamental_comp_masc,qt_tec_medio_fem,qt_tec_medio_masc,qt_tec_superior_fem,qt_tec_superior_masc,qt_tec_especializacao_fem,qt_tec_especializacao_masc,qt_tec_mestrado_fem,qt_tec_mestrado_masc,qt_tec_doutorado_fem,qt_tec_doutorado_masc,in_acesso_portal_capes,in_acesso_outras_bases,in_assina_outra_base,in_repositorio_institucional,in_busca_integrada,in_servico_internet,in_participa_rede_social,in_catalogo_online,qt_periodico_eletronico,qt_livro_eletronico,qt_doc_total,qt_doc_exe,qt_doc_ex_femi,qt_doc_ex_masc,qt_doc_ex_sem_grad,qt_doc_ex_grad,qt_doc_ex_esp,qt_doc_ex_mest,qt_doc_ex_dout,qt_doc_ex_int,qt_doc_ex_int_de,qt_doc_ex_int_sem_de,qt_doc_ex_parc,qt_doc_ex_hor,qt_doc_ex_0_29,qt_doc_ex_30_34,qt_doc_ex_35_39,qt_doc_ex_40_44,qt_doc_ex_45_49,qt_doc_ex_50_54,qt_doc_ex_55_59,qt_doc_ex_60_mais,qt_doc_ex_branca,qt_doc_ex_preta,qt_doc_ex_parda,qt_doc_ex_amarela,qt_doc_ex_indigena,qt_doc_ex_cor_nd,qt_doc_ex_bra,qt_doc_ex_est,qt_doc_ex_com_deficiencia,co_projeto,co_local_oferta,no_local_oferta
2021,Centro-Oeste,5,Mato Grosso,MT,51,Cuiab�,5103403,1,Centro-Sul Mato-g...,4,Cuiab�,17,1,1,FUNDACAO UNIVERSI...,1,1,UNIVERSIDADE FEDE...,UFMT,Avenida Fernando ...,2367,Cidade Universit�ria,Boa Esperan�a,78060900,1344,4,15,5,14,49,93,112,136,306,295,158,96,34,27,1,0,1,1,1,1,1,1,0,0,1718,1628,801,827,0,50,76,263,1239,1571,1275,296,57,0,58,214,310,335,242,160,137,172,1000,72,390,48,5,113,1593,35,6,2111801,105340,Campus Universit�...
2021,Centro-Oeste,5,Distrito Federal,DF,53,Bras�lia,5300108,1,Distrito Federal,1,Bras�lia,1,1,1,FUNDACAO UNIVERSI...,2,2,UNIVERSIDADE DE B...,UNB,Campus Universit�...,s/n,-,Asa Norte,70910900,3081,0,0,6,30,64,132,349,263,785,702,263,349,81,57,1,0,1,1,0,1,1,1,12438,493150,2942,2639,1174,1465,0,2,21,237,2379,2475,2255,220,164,0,38,160,323,474,431,359,375,479,1670,82,416,57,8,406,2496,143,10,2111801,4229,Campus Universit�...
2021,Nordeste,2,Sergipe,SE,28,S�o Crist�v�o,2806701,0,Leste Sergipano,3,Aracaju,11,1,1,FUNDACAO UNIVERSI...,3,3,UNIVERSIDADE FEDE...,UFS,Avenida Marechal ...,s/n,-,Jardim Rosa Elze,49100000,956,0,0,1,0,146,109,171,151,137,89,83,51,13,5,1,0,1,1,1,1,1,1,40534,316939,1833,1777,851,926,0,113,0,272,1392,1569,1404,165,208,0,57,163,317,354,289,233,191,173,172,35,269,9,2,1290,1747,30,4,2111801,1006435,Unidade SEDE
2021,Norte,1,Amazonas,AM,13,Manaus,1302603,1,Centro Amazonense,3,Manaus,7,1,1,FUNDACAO UNIVERSI...,4,4,UNIVERSIDADE FEDE...,UFAM,Av. Rodrigo Ot�vio,6200,Campus Universit�rio,Coroado II,69077000,1862,6,6,3,13,120,152,317,282,409,263,140,101,27,23,1,0,1,1,1,1,1,1,40191,3683,1981,1824,822,1002,0,111,141,436,1136,1730,1349,381,94,0,93,200,307,317,255,228,221,203,113,16,125,6,4,1560,1821,3,2,2111801,1036292,Unidade Sede
2021,Nordeste,2,Piau�,PI,22,Teresina,2211001,1,Centro-Norte Piau...,2,Teresina,3,1,1,FUNDACAO UNIVERSI...,14054,5,UNIVERSIDADE FEDE...,UFPI,Campus Universit�...,s/n,SG - 07,Ininga,64049550,1041,3,18,2,20,26,77,90,111,229,206,120,83,26,30,1,0,0,1,0,1,1,1,0,0,1577,1470,709,761,0,0,76,286,1108,1372,1182,190,98,0,27,156,280,286,197,149,149,226,110,42,204,0,2,1112,1470,0,0,2111801,102559,CAMPUS MINISTRO P...
2021,Sudeste,3,Minas Gerais,MG,31,Ouro Preto,3146107,0,Metropolitana de ...,7,Ouro Preto,33,1,1,UNIVERSIDADE FEDE...,6,6,UNIVERSIDADE FEDE...,UFOP,Rua Diogo de Vasc...,122,-,Centro,35400000,715,4,23,3,14,23,56,42,84,93,121,115,88,29,20,1,0,1,1,0,0,0,1,0,28762,928,846,332,514,0,2,16,82,746,839,790,49,7,0,3,67,194,199,117,97,93,76,402,25,146,4,1,268,844,2,3,2111801,664,Reitoria
2021,Sudeste,3,S�o Pa

1- Olhando para o estado de Minas Gerais, quantos municípios têm informações presentes
na base de dados?

In [10]:
spark.sql('''
    SELECT
        COUNT(co_municipio_ies) AS total_municipios
    FROM microdados_ies
    WHERE sg_uf_ies = 'MG'

''')

total_municipios
304


In [11]:
(
    df
    .where("sg_uf_ies == 'MG'")
    .agg(
        fn.count("co_municipio_ies").alias("total_municipios")
    )
)

total_municipios
304


2. Quantos professores doutores existem em cada cidade de Minas Gerais presente na
base de dados ?

In [17]:
spark.sql('''
     SELECT
         co_municipio_ies,
         no_municipio_ies,
        SUM(qt_doc_ex_dout) AS total_docentes_doutores
    FROM microdados_ies
    WHERE sg_uf_ies = 'MG'
    GROUP BY co_municipio_ies, no_municipio_ies
    ORDER BY total_docentes_doutores DESC
    
''')

co_municipio_ies,no_municipio_ies,total_docentes_doutores
3106200,Belo Horizonte,6615
3170206,Uberl�ndia,1939
3136702,Juiz de Fora,1915
3171303,Vi�osa,1185
3143302,Montes Claros,925
3170107,Uberaba,891
3138203,Lavras,812
3162500,S�o Jo�o del Rei,753
3146107,Ouro Preto,746
3121605,Diamantina,673


In [29]:
(
    df
    .where("sg_uf_ies = 'MG'")
    .groupby("co_municipio_ies","no_municipio_ies")
    .agg
    (
        fn.sum("qt_doc_ex_dout").alias("total_docentes_doutores")
    )
    .orderBy("total_docentes_doutores", ascending = False)

)

co_municipio_ies,no_municipio_ies,total_docentes_doutores
3106200,Belo Horizonte,6615
3170206,Uberl�ndia,1939
3136702,Juiz de Fora,1915
3171303,Vi�osa,1185
3143302,Montes Claros,925
3170107,Uberaba,891
3138203,Lavras,812
3162500,S�o Jo�o del Rei,753
3146107,Ouro Preto,746
3121605,Diamantina,673


3. Qual a quantidade de docentes com deficiência no estado do Paraná?

In [30]:
spark.sql("""
    SELECT
        SUM(qt_doc_ex_com_deficiencia) as total_docentes_PCD
    FROM microdados_ies
    WHERE sg_uf_ies = 'PR'


""")

total_docentes_PCD
160


In [31]:
(
    df
    .where("sg_uf_ies = 'PR'")
    .agg
    (
        fn.sum('qt_doc_ex_com_deficiencia').alias('total_docentes_PCD')
    )
)

total_docentes_PCD
160
